# Final Project

In [139]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#### Import data

In [140]:
Monday_df = pd.read_csv('archive_2/Monday_WorkingHours.csv')
Monday_df = pd.DataFrame(Monday_df)
print(Monday_df.shape) 
Monday_df.columns = Monday_df.columns.str.strip() #Remove extra spaces from column names

Tuesday_BruteForce = pd.read_csv('archive_2/Tuesday_BruteForce.csv')
Tuesday_BruteForce = pd.DataFrame(Tuesday_BruteForce)
print(Tuesday_BruteForce.shape)
Tuesday_BruteForce.columns = Tuesday_BruteForce.columns.str.strip()

Wednesday_DOS = pd.read_csv('archive_2/Wednesday_DOS_DDOS.csv')
Wednesday_DOS = pd.DataFrame(Wednesday_DOS)
print(Wednesday_DOS.shape)
Wednesday_DOS.columns = Wednesday_DOS.columns.str.strip() 

Thursday_WebAttack = pd.read_csv('archive_2/Thursday_Morning_WebAttacks.csv')
Thursday_WebAttack = pd.DataFrame(Thursday_WebAttack)
print(Thursday_WebAttack.shape)
Thursday_WebAttack.columns = Thursday_WebAttack.columns.str.strip()

Thursday_Infiltration = pd.read_csv('archive_2/Thursday_Afternoon_Infiltration.csv')
Thursday_Infiltration = pd.DataFrame(Thursday_Infiltration)
print(Thursday_Infiltration.shape)
Thursday_Infiltration.columns = Thursday_Infiltration.columns.str.strip()

Friday_Botnet = pd.read_csv('archive_2/Friday_Morning_Botnet.csv')
Friday_Botnet = pd.DataFrame(Friday_Botnet)
print(Friday_Botnet.shape)
Friday_Botnet.columns = Friday_Botnet.columns.str.strip()

Friday_Portscan = pd.read_csv('archive_2/Friday_Afternoon_PortScan.csv')
Friday_Portscan = pd.DataFrame(Friday_Portscan)
print(Friday_Portscan.shape)
Friday_Portscan.columns = Friday_Portscan.columns.str.strip()

Friday_DDOS = pd.read_csv('archive_2/Friday_Afternoon_DDos.csv')
Friday_DDOS = pd.DataFrame(Friday_DDOS)
print(Friday_DDOS.shape)
Friday_DDOS.columns = Friday_DDOS.columns.str.strip()

(529918, 79)
(445909, 79)
(692703, 79)
(170366, 79)
(288602, 79)
(191033, 79)
(286467, 79)
(225745, 79)


In [141]:
#columns = Monday_df.columns
#print(columns)

#### Drop destination port and create y df

In [142]:
#Monday
Mon_df_y = Monday_df[['Label']].copy()
print(Mon_df_y[:5])
Mon_drop_df = Monday_df.drop(['Destination Port', 'Label'], axis=1)
print(Mon_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(529918, 77)


In [143]:
#Tuesday Brute Force
BruteForce_df_y = Tuesday_BruteForce[['Label']].copy()
print(BruteForce_df_y[:5])
BruteForce_drop_df = Tuesday_BruteForce.drop(['Destination Port', 'Label'], axis=1)
print(BruteForce_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(445909, 77)


In [144]:
#Wednesday DOS
DOS_df_y = Wednesday_DOS[['Label']].copy()
print(DOS_df_y[:5])
DOS_drop_df = Wednesday_DOS.drop(['Destination Port', 'Label'], axis=1)
print(DOS_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(692703, 77)


In [145]:
#Thursday Web Attack
WebAttack_df_y = Thursday_WebAttack[['Label']].copy()
print(WebAttack_df_y[:5])
WebAttack_drop_df = Thursday_WebAttack.drop(['Destination Port', 'Label'], axis=1)
print(WebAttack_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(170366, 77)


In [146]:
#Thursday Infiltration
Infiltration_df_y = Thursday_Infiltration[['Label']].copy()
print(Infiltration_df_y[:5])
Infiltration_drop_df = Thursday_Infiltration.drop(['Destination Port', 'Label'], axis=1)
print(Infiltration_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(288602, 77)


In [147]:
#Friday Botnet
Botnet_df_y = Friday_Botnet[['Label']].copy()
print(Botnet_df_y[:5])
Botnet_drop_df = Friday_Botnet.drop(['Destination Port', 'Label'], axis=1)
print(Botnet_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(191033, 77)


In [148]:
#Friday Portscan
Portscan_df_y = Friday_Portscan[['Label']].copy()
print(Portscan_df_y[:5])
Portscan_drop_df = Friday_Portscan.drop(['Destination Port', 'Label'], axis=1)
print(Portscan_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(286467, 77)


In [149]:
#Friday DDOS
DDOS_df_y = Friday_DDOS[['Label']].copy()
print(DDOS_df_y[:5])
DDOS_drop_df = Friday_DDOS.drop(['Destination Port', 'Label'], axis=1)
print(DDOS_drop_df.shape)

    Label
0  BENIGN
1  BENIGN
2  BENIGN
3  BENIGN
4  BENIGN
(225745, 77)


#### Encode Labels

In [159]:
#Monday encoding
Mon_df_y['Label_encoding'] = Mon_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
Mon_df_y = Mon_df_y.drop(['Label'], axis=1)
print(Mon_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [161]:
#Tuesday Brute Force encoding
BruteForce_df_y['Label_encoding'] = BruteForce_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
BruteForce_df_y = BruteForce_df_y.drop(['Label'], axis=1)
print(BruteForce_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [162]:
#Wednesday DOS encoding
DOS_df_y['Label_encoding'] = DOS_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
DOS_df_y = DOS_df_y.drop(['Label'], axis=1)
print(DOS_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [163]:
#Thursday Web Attack encoding
WebAttack_df_y['Label_encoding'] = WebAttack_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
WebAttack_df_y = WebAttack_df_y.drop(['Label'], axis=1)
print(WebAttack_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [164]:
#Thursday Infiltration encoding
Infiltration_df_y['Label_encoding'] = Infiltration_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
Infiltration_df_y = Infiltration_df_y.drop(['Label'], axis=1)
print(Infiltration_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [165]:
#Friday Botnet encoding
Botnet_df_y['Label_encoding'] = Botnet_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
Botnet_df_y = Botnet_df_y.drop(['Label'], axis=1)
print(Botnet_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [166]:
#Friday Portscan encoding
Portscan_df_y['Label_encoding'] = Portscan_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
Portscan_df_y = Portscan_df_y.drop(['Label'], axis=1)
print(Portscan_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


In [ ]:
#Friday DDOS encoding
DDOS_df_y['Label_encoding'] = DDOS_df_y['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
DDOS_df_y = DDOS_df_y.drop(['Label'], axis=1)
print(DDOS_df_y[:5])

   Label_encoding
0               0
1               0
2               0
3               0
4               0


1    128027
0     97718
Name: Label_encoding, dtype: int64

#### Check that correct labels are maintained

In [175]:
print('Monday: ', '\n', Mon_df_y['Label_encoding'].value_counts())
print('BruteForce:', '\n',  BruteForce_df_y['Label_encoding'].value_counts())
print('DOS: ', '\n', DOS_df_y['Label_encoding'].value_counts())
print('WebAttack: ', '\n', WebAttack_df_y['Label_encoding'].value_counts())
print('Infiltration: ', '\n', Infiltration_df_y['Label_encoding'].value_counts())
print('Botnet: ', '\n', Botnet_df_y['Label_encoding'].value_counts())
print('Portscan: ', '\n', Portscan_df_y['Label_encoding'].value_counts())
print('DDOS: ', '\n', DDOS_df_y['Label_encoding'].value_counts())

Monday:  
 0    529918
Name: Label_encoding, dtype: int64
BruteForce: 
 0    432074
1     13835
Name: Label_encoding, dtype: int64
DOS:  
 0    440031
1    252672
Name: Label_encoding, dtype: int64
WebAttack:  
 0    168186
1      2180
Name: Label_encoding, dtype: int64
Infiltration:  
 0    288566
1        36
Name: Label_encoding, dtype: int64
Botnet:  
 0    189067
1      1966
Name: Label_encoding, dtype: int64
Portscan:  
 1    158930
0    127537
Name: Label_encoding, dtype: int64
DDOS:  
 1    128027
0     97718
Name: Label_encoding, dtype: int64


#### Train test split all data

In [ ]:
#Monday train, test, val split
X_Mon_temp, X_Mon_test, Y_Mon_temp, Y_Mon_test = train_test_split(
    Mon_drop_df, Mon_df_y, 
    test_size=.2, 
    stratify=Mon_df_y, 
    random_state=28)
X_Mon_train, X_Mon_val, Y_Mon_train, Y_Mon_val = train_test_split(
    X_Mon_temp, Y_Mon_temp, 
    test_size=.125, 
    stratify=Y_Mon_temp, 
    random_state=28)

In [ ]:
#BruteForce train, test, val split
X_BruteForce_temp, X_BruteForce_test, Y_BruteForce_temp, Y_BruteForce_test = train_test_split(
    BruteForce_drop_df, BruteForce_df_y, 
    test_size=.2, 
    stratify=BruteForce_df_y, 
    random_state=28)
X_BruteForce_train, X_BruteForce_val, Y_BruteForce_train, Y_BruteForce_val = train_test_split(
    X_BruteForce_temp, Y_BruteForce_temp, 
    test_size=.125,  
    stratify=Y_BruteForce_temp, 
    random_state=28)

In [ ]:
#DOS train, test, val split
X_DOS_temp, X_DOS_test, Y_DOS_temp, Y_DOS_test = train_test_split(
    DOS_drop_df, DOS_df_y, 
    test_size=.2, 
    stratify=DOS_df_y, 
    random_state=28)
X_DOS_train, X_DOS_val, Y_DOS_train, Y_DOS_val = train_test_split(
    X_DOS_temp, Y_DOS_temp, 
    test_size=.125,  
    stratify=Y_DOS_temp, 
    random_state=28)

In [ ]:
#WebAttack train, test, val split
X_WebAttack_temp, X_WebAttack_test, Y_WebAttack_temp, Y_WebAttack_test = train_test_split(
    WebAttack_drop_df, WebAttack_df_y, 
    test_size=.2, 
    stratify=WebAttack_df_y, 
    random_state=28)
X_WebAttack_train, X_WebAttack_val, Y_WebAttack_train, Y_WebAttack_val = train_test_split(
    X_WebAttack_temp, Y_WebAttack_temp, 
    test_size=.125,  
    stratify=Y_WebAttack_temp, 
    random_state=28)

In [ ]:
#Infiltration train, test, val split
X_Infiltration_temp, X_Infiltration_test, Y_Infiltration_temp, Y_Infiltration_test = train_test_split(
    Infiltration_drop_df, Infiltration_df_y, 
    test_size=.2, 
    stratify=Infiltration_df_y, 
    random_state=28)
X_Infiltration_train, X_Infiltration_val, Y_Infiltration_train, Y_Infiltration_val = train_test_split(
    X_Infiltration_temp, Y_Infiltration_temp, 
    test_size=.125,  
    stratify=Y_Infiltration_temp, 
    random_state=28)

In [ ]:
#Botnet train, test, val split
X_Botnet_temp, X_Botnet_test, Y_Botnet_temp, Y_Botnet_test = train_test_split(
    Botnet_drop_df, Botnet_df_y, 
    test_size=.2, 
    stratify=Botnet_df_y, 
    random_state=28)
X_Botnet_train, X_Botnet_val, Y_Botnet_train, Y_Botnet_val = train_test_split(
    X_Botnet_temp, Y_Botnet_temp, 
    test_size=.125,  
    stratify=Y_Botnet_temp, 
    random_state=28)

In [ ]:
#Portscan train, test, val split
X_Portscan_temp, X_Portscan_test, Y_Portscan_temp, Y_Portscan_test = train_test_split(
    Portscan_drop_df, Portscan_df_y, 
    test_size=.2, 
    stratify=Portscan_df_y, 
    random_state=28)
X_Portscan_train, X_Portscan_val, Y_Portscan_train, Y_Portscan_val = train_test_split(
    X_Portscan_temp, Y_Portscan_temp, 
    test_size=.125,  
    stratify=Y_Portscan_temp, 
    random_state=28)

In [ ]:
#Portscan train, test, val split
X_DDOS_temp, X_DDOS_test, Y_DDOS_temp, Y_DDOS_test = train_test_split(
    DDOS_drop_df, DDOS_df_y, 
    test_size=.2, 
    stratify=DDOS_df_y, 
    random_state=28)
X_DDOS_train, X_DDOS_val, Y_DDOS_train, Y_DDOS_val = train_test_split(
    X_DDOS_temp, Y_DDOS_temp, 
    test_size=.125,  
    stratify=Y_DDOS_temp, 
    random_state=28)